# IoT Graph

This demo is designed to demonstrate the some of the additional insights that you can gain by loading your existing streaming IoT data into a graph database.

This demo will exist in multiple parts:

1. Converting you existing data to be loaded into the graph
2. Enriching your data with graph attributes
3. Extracting additional insights from your data with graph queries

## The Goal

The goal of this setup is to attempt to recognize how air and thermal energy flow throughout an indoor space. This indoor space is a 3rd floor apartment in Boston, MA consisting of two rooms, a bedroom and a workshop. Both rooms have a window, the one in the workshop faces East, and the one in the Bedroom faces South. 

In each window, there is a custom built sensor package. Each sensor records the ambient Temperature, Humidity, Pressure, and amount of Light at their location by their corresponding window. Additionally, the outdoor values for each of those readings are recorded as well.

<img src="https://i.ibb.co/tLyYYMZ/Sensor-1.jpg" width="44%">
<img src="https://i.ibb.co/gPRQKHm/Sensor-2.jpg" width="40%">

The hope is that we will be able to predict the readings of these sensors given a weather forecast for any given day. Additionally, we will be exploring the relationship between the sensors and attempt to identify activities that are happening in the rooms based on the sensor readings. Finally, the real world isn't perfect, so we have some gaps in our data and it would be nice to be able to identify and fill in those values with predictions.



# Part 1 - Understanding the Data and Building the Graph

## Setting up the TigerGraph server

Head over to [tgcloud.io](tgcloud.io) and create an account to begin creating you first TigerGraph solution.

![](https://i.ibb.co/synTScQ/Screen-Shot-2021-08-24-at-5-23-52-PM.png)

Once you have created your account and signed in, you can provision a solution by clicking the **Create Solution** button. 

![](https://i.ibb.co/vjwyp4T/Screen-Shot-2021-07-28-at-12-13-14-AM.png)

Next, select **Blank** from the Starter Kits menu. We'll be setting up our own graph here. Click the next button at the bottom to continue to the **Instance Settings**.

![](https://i.ibb.co/MSkwPBk/Screen-Shot-2021-07-28-at-12-21-10-AM.png)

We'll use a free tier instance for this demo, you can leave the rest of these settings default. Click the next button at the bottom to move on to setting up the **Solution Settings**.

![](https://i.ibb.co/kGXFjvD/Screen-Shot-2021-07-28-at-12-24-07-AM.png)

The last things that we'll need to customize are the instance settings. Here's what each one does:
- **Solution Name** - the name you will see this solution referred to on the **My Solutions** page
- **Solution Tag** - single word tags that can be used to filter the solution in the **My Solutions** page
- **Initial Password** - the password used to log into this TigerGraph Solution - default: `tigergraph`
- **Subdomain** - the subdomain where your TigerGraph Solution will live - can only conatin lower case letters, numbers, and hyphens **(Set this to something you will remember as you will use it to access your solution)**
- **Description** - a description of the solution


#### For this demo, you can use:
- **Name** - `IoT Graph`
- **Tag** - `iot`
- **Password** - `tigergrpah`
- **Subdomain** - `iot-graph`
- **Description** - you can leave blank

![](https://i.ibb.co/gtCmXcm/Screen-Shot-2021-07-28-at-12-27-19-AM.png)

Finally, review your settings on the **Confirmation Page** and click submit if they look good.

![](https://i.ibb.co/hyN2Dcv/Screen-Shot-2021-07-28-at-12-31-12-AM.png)

It will take about 5 minutes for your solution to provision and spin up for the first time.

## The Data

IoT data can take a variety of forms. It can be streaming data in any varitey of formats: websockets, kafka streams, data dumps and chron jobs, there's really no limit to how the real world can structure "realtime" IoT data.

For this example, I wrote a library to allow ESP32 microcontrollers to post directly to Tigergraph. The pre-alpha version of this library is available here: https://github.com/DanBarkus/microTigerGraph Additionally, this could be set up using our Node Red style interface: [TigerFlow](https://github.com/TigerGraph-DevLabs/TigerFlow)

![](https://i.ibb.co/0CkZFgJ/Screen-Shot-2021-09-23-at-11-46-45-AM.png)

This basic diagram can be split in half for easier understanding, but demsonstrates the basic method that we will want store our data in. 

On the left half, we have information describing the origins of any given `Reading`. A `Device` is the physical sensor module that recorded a `Reading`. We have two physical devices, **Window** and **Workshop** (the sensors) and one virtual device **Outside**.

Each device has four different `Reading_Type`s that it can capture: **Temperature**, **Humidity**, **Pressure**, and **Light Intensity**. With this structure, we can dynamically select any type of reading from any particular device.

On the right side is our information about when the reading was captured. This will come in handy when we're looing to corelate readings from multiple sensors and attempt to predict missing or upcoming data.

`Day` is a datetime representing the Day of the year that the `Reading` was captured on. `Hour` is an INT representing the hour of the day when a `Reading` was captured. We could go down to `Minute` here, but the frequency of our data doesn't necessitate storage more granular than an hour.

### Exploring the Data
Let's take a look at what this data actually looks like.
Because the data for this demo was generated by streaming devices, there's no easy way for me to give you access to that streaming data. Instead I've included about a 3 week export of a portion of the sensor data.

In [ ]:
# !git clone https://github.com/DanBarkus/IoT-Dashboard.git

That data exists in 3 files, one for each of our sensors.

In [ ]:
!ls IoT-Dashboard/data/

Those files are structured like this:

|@device|@rdgType|value|type|captured_at|
|-------|--------|-----|----|-----------|
|Device Name (corresponds to `Device` vertex id)|Reading Type (corresponds to `Reading_Type` vertex id)|Value of Reading|Type of sensor that produced the reading|`DATETIME` that the reading was captured at|

In [ ]:
!head IoT-Dashboard/data/Window.csv

A keen observer may notice that we're missing a couple things from the schema picture shown earlier. Mainly, there's no field for just `Date` or `Hour`. That's okay because we'll be extracting those from the **captured_at** value during data loading.

You may also notice that there are no unique identifers for our rows of data (the first column has numbers, but they are duplicated across our three data files an were purely a product of the data extraction process). We will also be generating a unique identifier for each reading based on a combination of its `Device`, `Reading_Type` and `captured_at` values.

## Connecting from Python

### Python Installs and Imports for **TigerGraph**

pyTigerGraph will be the main tool that we'll be using to interface with the graph from python. You can learn more about pyTigerGraph with our [intro video](https://www.youtube.com/watch?v=2BcC3C-qfX4&t=1s), or by reading the [docs](https://pytigergraph.github.io/pyTigerGraph/).

In [ ]:
# !pip install pyTigerGraph

In [ ]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)
def pprint(input):
  print(json.dumps(input, indent=2))

### Establishing a Connection

Change the values below to so that `hostName` matches your **Subdomain** and the `password` matches what you set if you've changed it.

This will establish the inital connection to our TigerGraph solution. There is currently no graph or schema in the solution, so that will need to be created.

In [ ]:
# Connection parameters
# hostName = "https://iot-2.i.tgcloud.io"
hostName = "http://homelab-k3s.172.16.17.16.nip.io"
userName = "tigergraph"
password = "Tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

## Understanding the Schema

The core of any graph solution is a **Schema**. The **Schema** defines what each type of vertex is and how it relates to other types of vertices via edges.

Both Vertices and Edges can have attributes which can further describe them.

Think of each **Vertex Type** as a **Table** in a relational database. **Attributes** of that Vertex Type are your **Columns** from that Table and each **Individual Vertex** is analagous to a **Row** from your Table. **Edges** represent **Joins** between tables. As part of the Schema, Edges are computed during data load and sotred in memory rather than being computed during query time.

This is an absolute bare-bones manner of implementing a graph schema, but since we're working on converting our existing relational data to graph data, it makes sense to start with something familiar and build off of that.

This is the schema that we'll be starting with:
![](https://i.ibb.co/0CkZFgJ/Screen-Shot-2021-09-23-at-11-46-45-AM.png)

You'll notice that our data files already contain:
- `Reading` - with value and addtional information
- `Device` - that created the Reading
- `Reading Type` - that the Reading represents
- Full DATETIME that the reading was captured at

That means that there's no explicit Column in our Data for `Day` or `Hour`. That's okay because they're both contained in the `captured_at` value of each `Reading`.

### The Story Behind `Day` and `Hour`

`Day` and `Hour` might not make sense at first as Vertex Types. Why not just store the full datetime attached to each `Reading`?

The reason comes down to our data. Each of the indoor sensors captures a new batch of readings about every 2 minutes. The Outdoor data is fetched from a weather API and has 15 minute to 1 hour intervals seemingly at random. 

If we're looking to correlate our Outdoor readings with our Indoor readings then we can't compare them 1:1 due to the difference in frequency (max 1 hour vs 2 minutes).

The two options are:
- Interpolate the Outdoor readings to the same frequency as the Indoor readings
  - Pros: High frequency data from both sources
  - Cons: Assumed values of interpolated readings can yield inaccurate results
- Round Indoor (and Outdoor) readings to the same low frequency that contains at least one reading per sensor, per period
  - Pros: More accurate as readings aren't assumed
  - Cons: Low frequency of data

For this demo, I chose the second option, Round to a common value. This is why `Hour` exists. When comparing readings from Outdoor and Indoor, the query can simply SELECT any `Reading` vertices that are connected input `Day`, `Hour`, and `Device` vertices.

`Hour` and `Day` essentially act as an easily accessable way for us to query a specific time period without having to check the date of every `Reading` in the Graph. **VertexID** exists entirely in **RAM** and **Edges** act as **Pointers** between memory addresses. **Attributes** are stored on **Disk**, so accessing them incurs a performance penalty. **Without** `Hour` and `Day`, selecting all `Readings` from an hour long time period on a specific day would involve selecting every `Reading` in the graph and checking it's captured_at attribute on disk to see if it is within the time range. **With** `Hour` and `Day`, we use the **VertexID** (in RAM) of the input `Day` and `Hour` to traverse Edges (also in RAM) to any `Reading` vertices connected to both the input `Hour` and `Day`. With this method, the entire query operates on data stored in RAM and only touches the verties that are actually returned from the query.

At the scale of the data in this demo, the performance impact of these methods is negligable (<3ms), but real-world scale with hundreds of millions+ readings will benefit from foresight when building the schema.

## Loading the Schema

We'll be creating the schema through a series of **GSQL** queries that we'll execute through pyTigerGraph.

The **schema** describes **each type of node in the graph and how it relates to other types of nodes via edges**. You can learn more about the GSQL commands for schema definition [in our docs](https://docs.tigergraph.com/start/gsql-101/define-a-schema). But in a nutshell it looks like this:

### Create Vertices

`CREATE VERTEX <VERTEX_TYPE_NAME>(PRIMARY_ID <PRIMARY_ID_NAME> <PRIMARY_ID_TYPE>, <ATTRIBUTE_NAME> <ATTRIBUTE_TYPE>, ...)`

- `CREATE` - We're adding something to the solution
- `VERTEX` - It's a vertex type that's being added
- `<VERTEX_TYPE_NAME>` - The name you want to give to your vertex type
- `PRIMARY_ID` - Specifies that the next input will be the **name of the primary_id**
- `<PRIMARY_ID_NAME>` - The name of the primary_id field
- `<PRIMARY_ID_TYPE>` - The variable type of the primary_id
- `<ATTRIBUTE_NAME>` - The name of an attribute of the vertex - **Attributes are optional and each vertex can contain more than one**
- `<ATTRIBUTE_TYPE>` - The variable type of the attribute

Addtionally, we want to specify that the `primary_id` of the vertex should also be duplicated as an attribute so that we can reference it from queries. That is done by adding:
`WITH primary_id_as_attribute="true"`
to the end of the VERTEX decleration. 

In [ ]:
conn.gsql('''
drop graph IoTDashboard
create graph IoTDashboard()
use graph IoTDashboard
drop job iot_schema
create schema_change job iot_schema for graph IoTDashboard {
ADD VERTEX Reading(PRIMARY_ID id STRING, value FLOAT, type STRING, captured_at DATETIME) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
ADD VERTEX Device(PRIMARY_ID id STRING, type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
ADD VERTEX Reading_Type(PRIMARY_ID id STRING, unit STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
ADD VERTEX Day(PRIMARY_ID date STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
ADD VERTEX Hour(PRIMARY_ID hour INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
add DIRECTED EDGE has_reading(FROM Device, TO Reading) WITH REVERSE_EDGE="reverse_has_reading";
add DIRECTED EDGE on_day(FROM Reading, TO Day) WITH REVERSE_EDGE="reverse_on_day";
add DIRECTED EDGE on_hour(FROM Reading, TO Hour) WITH REVERSE_EDGE="reverse_on_hour";
add DIRECTED EDGE of_type(FROM Reading, TO Reading_Type) WITH REVERSE_EDGE="reverse_of_type";
}
''')

### Create Edges

`CREATE DIRECTED|UNDIRECTED EDGE <EDGE_TYPE_NAME>(FROM <SOURCE_VERTEX_TYPE>, TO <DESTINATION_VERTEX_TYPE>, <ATTRIBUTE_NAME> <ATTRIBUTE_TYPE>, ...)`

- `CREATE` - We're adding something to the solution
- `DIRECTED|UNDIRECTED` - Specify if the edge is Directed or Undirected
- `EDGE` - It's an edge type that's being added
- `<EDGE_TYPE_NAME>` - The name you want to give to your edge type
- `FROM` - Specifies that the next input will be the **source vertex type**
- `<SOURCE_VERTEX_TYPE>` - The vertex type of the source of the edge
- `TO` - Specifies that the next input will be the **destination vertex type**
- `<DESTINATION_VERTEX_TYPE>` - The vertex type of the destination of the edge
- `<ATTRIBUTE_NAME>` - The name of an attribute of the edge - **Attributes are optional and each edge can contain more than one**
- `<ATTRIBUTE_TYPE>` - The variable type of the attribute

In [ ]:
# conn.gsql('''
# CREATE DIRECTED EDGE has_reading(FROM Device, TO Reading) WITH REVERSE_EDGE="reverse_has_reading"
# CREATE DIRECTED EDGE on_day(FROM Reading, TO Day) WITH REVERSE_EDGE="reverse_on_day"
# CREATE DIRECTED EDGE on_hour(FROM Reading, TO Hour) WITH REVERSE_EDGE="reverse_on_hour"
# CREATE DIRECTED EDGE of_type(FROM Reading, TO Reading_Type) WITH REVERSE_EDGE="reverse_of_type"
# ''')

## Create Graph

The schema that we just created exists in the **Global** sense. We can have more than one **Graph** per **Solution**, so a **Global** schema allows us to re-use parts or all of that schema across multiple graphs.

We're not going to need to do anything fancy like that for this demo, it's just important to know that any **Graphs** can contain none or any number of elements from the **Global** schema and can even contain elements that are unique to that **Graph's** schema.

The command to create the graph is another GSQL block following this pattern:

`CREATE GRAPH <GRAPH_NAME>(<VERTEX_TYPE>|<EDGE_TYPE>, ...)`

- `CREATE` - We're adding something to the solution
- `GRAPH` - It's an graph that's being added
- `<GRAPH_NAME>` - The name you want to give to your graph
- `<VERTEX_TYPE>|<EDGE_TYPE>` - The name of a **Vertex Type** or **Edge Type**  - This can be repeated for as many vertex types or edge type you want to include in your graph

In [ ]:
# conn.gsql('''
# CREATE GRAPH IoTDashboard(Reading, Device, Reading_Type, Day, Hour, has_reading, on_day, on_hour, of_type)
# ''')
conn.gsql('''
use graph IoTDashboard
run schema_change job iot_schema
''')

### Re-connecting to the Graph

Now that the graph is created, we need to update our pyTigerGraph connection to point specifically to our graph. This will allow us to create a **Secret**, then use that **Secret** to get a **Token** which will be used for secure authentication to that specific graph in our **Solution**. 

In [ ]:
graphName = "IoTDashboard"
conn.graphname = graphName
# secret = conn.createSecret()
# token = conn.getToken(secret, setToken=True)
token = "cdepgpk5oebgs6gsqbvijccb3lngavbg"
conn.apiToken = token
print(token)

## Loading Jobs

A **Loading Job** defines how fields from our input files map to the primary_id and attributes of vertices and source and destination vertices and attributes of edges.

To help visualize what's going on here, let's clone our data so we know what it looks like.

### Anatomy of a Loading Job

Looking at the loading job below, you'll see that we don't refer to the column names of our data by their headers, but rather by their column number. The chart below shows how that compares to the header from our alergies file above.

|Column Number|\$0|\$1|\$2|\$3|\$4|\$5|\$6|
|---|---|---|---|---|---|---|---|
|Column Name|ID|START|STOP|PATIENT|ENCOUNTER|CODE|DESCRIPTION|

Go ahead and creat this loading job and I'll break down exactly what's going on here afterwards.

In [ ]:
!head IoT-Dashboard/data/Outside.csv

In [ ]:
conn.gsql('''
CREATE LOADING JOB load_sensor FOR GRAPH IoTDashboard {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Reading VALUES(gsql_concat($1,$2,$5), $3, $2, $5) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO VERTEX Device VALUES($1, _) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO VERTEX Reading_Type VALUES($2, _) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE of_type VALUES(gsql_concat($1,$2,$5), $2) USING SEPARATOR=",", HEADER="true", EOL="\n";
      LOAD MyDataSource TO EDGE has_reading VALUES($1, gsql_concat($1,$2,$5)) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }
''')

#### Loading Job Breakdown

`CREATE LOADING JOB <LOADING_JOB_NAME> FOR GRAPH <GRAPH_NAME> {`

- `CREATE` - We're adding something to the solution
- `LOADING JOB` - It's an loading job that's being added
- `<LOADING_JOB_NAME>` - The name you want to give to your loading job
- `FOR GRAPH` - The next input will specify which graph the job is for
- `<GRAPH_NAME>` - The name of the graph that the loading job is for

```
DEFINE FILENAME <FILE_VARIABLE_NAME>;
LOAD <FILE_VARIABLE_NAME>
  TO VERTEX|EDGE <VERTEX_TYPE>|<EDGE_TYPE> VALUES(<COLUMN_NUMBER>, ...)
```

- `DEFINE` - Defining a Variable
- `FILENAME` - The type of variable being defined
- `<FILE_VARIABLE_NAME>` - The name of the variable that will represent our input file
- `LOAD` - Specify that the next input is the file that we will be loading
- `<FILE_VARIABLE_NAME>` - The file variable that we are loading
- `TO` - The next input is a vertex type or edge type that the loading job will apply to
- `VERTEX|EDGE` - Specify if the Job is loading to a Vertex or Edge
- `<VERTEX_TYPE>|<EDGE_TYPE>` - The name of the vertex type or edge type being loaded into
- `VALUES` - The next input contains the Column Numbers in order of the fields they represent

**Values Layout**

**Vertex**
`VALUES(PRIMARY_ID, ATTRIBUTE_1, ATTRIBUTE_2, ...)`

**Edge**
`VALUES(SOURCE_ID, DESTINATION_ID, ATTRIBUTE_1, ATTRIBUTE_2, ...)`

Additionally we specify any additional options for the loading job after a `USING` statement. For a full list of options check the [Loading Job Documentation](https://docs.tigergraph.com/dev/gsql-ref/ddl-and-loading/creating-a-loading-job#create-loading-job)

## Loading Data

Now that the loading jobs have been created, we can begin actually loading in data. We'll be stepping away from the GSQL heavy work that we've been using so far and switch back to more python oriented code for loading.

First, we load the data file into a variable.

`uploadFile()` requires 3 inputs:
- `filePath` - The actual data file to load
- `fileTag` - This is the name of the variable that the file will correspond to in the loading job. If you remember, we're using `f1` as our FILENAME variable in the loading jobs.
- `jobName` - The name of the corresponding loading job to run

In [ ]:
import os

# Run each of our 3 data files through the Loading Job
directory = r'IoT-Dashboard/data'
for dataFile in os.scandir(directory):
  if dataFile.path.endswith(".csv"):
    results = conn.uploadFile(dataFile, fileTag='MyDataSource', jobName='load_sensor')
    print(json.dumps(results, indent=2))

## Add Hours and Days

Remember those `Hour` and `Day` vertices that we didn't end up loading in with the rest of our data? Well now it's time to create them. 

We're doing this from a query rather than when loading the data right now. In a real world use-case, you would want these edges to be generated when the readings are added to the graph. Because we're going through the process of upgrading our existing solution, we have data that we need to generate these edges for.

In [ ]:
conn.gsql(
    '''
    USE GRAPH IoTDashboard
    CREATE QUERY addHoursAndDays() FOR GRAPH IoTDashboard { 
      MinAccum<INT> @currentHour;
      MaxAccum<STRING> @currentDay;
      readings = {Reading.*};
      
      noHour = SELECT r FROM readings:r
        WHERE
          r.OUTDEGREE("on_hour") == 0
        ACCUM
          r.@currentHour = HOUR(r.captured_at)
        POST-ACCUM
          INSERT INTO on_hour (FROM, TO)VALUES (r.id, r.@currentHour);
      
      noDay = SELECT r FROM readings:r
        WHERE
          r.OUTDEGREE("on_day") == 0
        ACCUM
          r.@currentDay += datetime_format(r.captured_at, "%Y-%m-%d")
        POST-ACCUM
          INSERT INTO on_day (FROM, TO)VALUES (r.id, r.@currentDay);
      
      PRINT noDay;
    }
    INSTALL QUERY addHoursAndDays
    '''
)


In [ ]:
conn.runInstalledQuery('addHoursAndDays', params={})

# Part 1.5 - Building the Dashboard

The graph has been set up and the data has been loaded. We could start poking away at the data in Graph Studio. Or... hear me out, we could build out a cool dashboard to show off the data.

I'm glad you picked the dashboard, I did too.

We'll use [Plotly Dash](https://dash.plotly.com/introduction) to build out the interface.

## Installing Dash

Let's get the installs and imports out of the way first.

In [ ]:
!pip install -q jupyter-dash
!pip install -q plotly
!pip install -q dash-bootstrap-components

In [ ]:
from jupyter_dash import JupyterDash
import datetime
import math
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Creating Queries to Fetch Data from the Graph

I know we're supposed to be working on the dashboard, but the dashboard isn't much use if it can't get to our data. Further, we need to know what that data coming back from our queries looks like before we can start writing code to display it.

### Get all Readings for a Sensor

At the core of our dashboard, we need to display our data. We'll have a couple charts that we'll be displaying and they'll each have a dropdown to select which information will be displayed.

We can either fetch the relevant information each time that dropdown is changed, or we can fetch all the data at once when the page loads and just filter on the front end from there. Will be doing the later option as it reduces the number of total calls our page will need to make and reduces the overall time spent loading.

In [ ]:
conn.gsql(
    '''
    USE GRAPH IoTDashboard
    CREATE QUERY getReadingsForSensor(VERTEX<Device> inDevice) FOR GRAPH IoTDashboard SYNTAX v2 {
      STRING visVar;
      device = {inDevice};
      
      IF inDevice.id == "Outside" 
        THEN visVar = "CLOUD";
      ELSE
        visVar = "LUX";
      END;
      
      temperatures = SELECT t FROM device - () - Reading:t - (of_type>) - Reading_Type:rt WHERE
        rt.id == "TEMPERATURE";
      
      PRINT temperatures;
      
      humidities = SELECT t FROM device - () - Reading:t - (of_type>) - Reading_Type:rt WHERE
        rt.id == "HUMIDITY";
      
      PRINT humidities;
      
      pressures = SELECT t FROM device - () - Reading:t - (of_type>) - Reading_Type:rt WHERE
        rt.id == "PRESSURE";
      
      PRINT pressures;
      
      lux = SELECT t FROM device - () - Reading:t - (of_type>) - Reading_Type:rt WHERE
        rt.id == visVar;
      
      PRINT lux;
    }
    '''
)

In [ ]:
conn.gsql('''
INSTALL QUERY getReadingsForSensor
''')

### Get NEW Readings for a Sensor

This is realtime data (or at least it is on the original graph...) so the dashboard needs to be able to fetch updated data if its available.

This query will only return the most recent readings for an input device.

In [ ]:
conn.gsql(
    '''
    USE GRAPH IoTDashboard
    CREATE QUERY getMostRecentForSensor(VERTEX<Device> inDevice) FOR GRAPH IoTDashboard SYNTAX v2{ 
      STRING visVar;
      MapAccum<STRING, FLOAT> @@mostRecent;
      SetAccum<VERTEX> @@tempReadings;
      SetAccum<VERTEX> @@humidReadings;
      SetAccum<VERTEX> @@pressureReadings;
      SetAccum<VERTEX> @@luxReadings;
      MaxAccum<DATETIME> @@recentTime;
      
      dev = {inDevice};
      
      IF inDevice.id == "Outside" 
        THEN visVar = "CLOUD";
      ELSE
        visVar = "LUX";
      END;
      
      
      temperature = SELECT r FROM dev:l - (:e) - Reading:r - () - Reading_Type:t WHERE t.id == "TEMPERATURE"
          ACCUM
            @@recentTime += r.captured_at
          POST-ACCUM
            IF r.captured_at == @@recentTime THEN
              @@mostRecent += ("TEMPERATURE" -> r.value),
              @@tempReadings += r
            END;
      
      @@recentTime = to_datetime("0");
      
      humidity = SELECT r FROM dev:l - (:e) - Reading:r - () - Reading_Type:t WHERE t.id == "HUMIDITY"
          ACCUM
            @@recentTime += r.captured_at
          POST-ACCUM
            IF r.captured_at == @@recentTime THEN
              @@mostRecent += ("HUMIDITY" -> r.value),
              @@humidReadings += r
            END;
      
      @@recentTime = to_datetime("0");
      
      pressure = SELECT r FROM dev:l - (:e) - Reading:r - () - Reading_Type:t WHERE t.id == "PRESSURE"
          ACCUM
            @@recentTime += r.captured_at
          POST-ACCUM
            IF r.captured_at == @@recentTime THEN
              @@mostRecent += ("PRESSURE" -> r.value),
              @@pressureReadings += r
            END;
      
      @@recentTime = to_datetime("0");
      
      lux = SELECT r FROM dev:l - (:e) - Reading:r - () - Reading_Type:t WHERE t.id == visVar
          ACCUM
            @@recentTime += r.captured_at
          POST-ACCUM
            IF r.captured_at == @@recentTime THEN
              @@mostRecent += ("LUX" -> r.value),
              @@luxReadings += r
            END;
      
      temperatures = {@@tempReadings};
      humidities = {@@humidReadings};
      pressures = {@@pressureReadings};
      lux = {@@luxReadings};

      print temperatures;
      print humidities;
      print pressures;
      print lux;
      print @@mostRecent;
      print @@recentTime;
    }
    INSTALL QUERY getMostRecentForSensor
    '''
)

## The Dashboard for Real this Time

Now it's time to actually build out the Dash dashboard and related functions. We'll start again with the functions needed to fetch our data. We wrote the query, now we need the python function that will actually call that query and process the results into the format we want.

### Global Variables

In [ ]:
sensors = ["Window", "Workshop", "Outside"]
readingTypes = ["Temperature", "Humidity", "Pressure", "Lux"]
currentSensor = ""
currentType = ""
data = {}

### Data Retrieval and Formatting from TigerGraph

This function serves as the backbone for retreiving reading data from TigerGraph. There's two ways that we want to get readings data, and we want them both to return in the same format.

1. We first want ALL readings, this is on page load and is the initial data fetch.
2. After we have all the data, we need to check periodically to see if there's new data and only fetch the new data (no sense wasting resources processing the full data again).

This function looks like a lot of code, but it's actually pretty simple (just done 4x for each reading type)

Here's the basic breakdown:

1. Fetch data from TigerGraph
  - `conn.runInstalledQuery("getReadingsForSensor")` - query returns an list of 4 vertex sets, one for each reading type containing all readings of that type for the input sensor
2. Convert each vertex set to a Pandas dataframe
  - `conn.vertexSetToDataFrame()`
3. Convert `captured_at` from a string to a python datetime
4. Order by `captured_at`
5. Set the `type` based on the reading type

In [ ]:
# Run the Query and return a dataframe for each `Reading_Type` returned
def pickData(sensor, firstTime=False):
  if firstTime:
    res = conn.runInstalledQuery("getReadingsForSensor", params={"inDevice": sensor})
  else:
    res = conn.runInstalledQuery("getMostRecentForSensor", params={"inDevice": sensor})
  temps = res[0]["temperatures"]
  humids = res[1]["humidities"]
  pressures = res[2]["pressures"]
  lux = res[3]["lux"]

  # convert the vertex sets from the query in to Pandas Dataframes
  dfTemps = conn.vertexSetToDataFrame(temps)
  dfHumids = conn.vertexSetToDataFrame(humids)
  dfPressures = conn.vertexSetToDataFrame(pressures)
  dfLux = conn.vertexSetToDataFrame(lux)

  # Convert captured_at to Datetime
  dfTemps['captured_at'] = pd.to_datetime(dfTemps['captured_at'])
  # Rename the type (it was incorrectly set on the sensors)
  dfTemps["type"] = dfTemps["id"].str.extract(r'([A-Z]{2,})')
  # Sort by date
  dfTemps= dfTemps.sort_values("captured_at")

  dfHumids['captured_at'] = pd.to_datetime(dfHumids['captured_at'])
  dfHumids["type"] = dfHumids["id"].str.extract(r'([A-Z]{2,})')
  dfHumids= dfHumids.sort_values("captured_at")

  dfPressures['captured_at'] = pd.to_datetime(dfPressures['captured_at'])
  dfPressures["type"] = dfPressures["id"].str.extract(r'([A-Z]{2,})')
  dfPressures= dfPressures.sort_values("captured_at")
  if sensor != "Outside":
    dfPressures["value"] /= 100  #divide by 100 to convert from Pascals to mBar

  dfLux['captured_at'] = pd.to_datetime(dfLux['captured_at'])
  dfLux["type"] = dfLux["id"].str.extract(r'([A-Z]{2,})')
  dfLux= dfLux.sort_values("captured_at")

  return dfTemps,dfHumids,dfPressures,dfLux
# pickData("Outside")

#### `getData()` - Gets All Data

Calls the above function and sets the contents of `data` to the returned data

In [ ]:
# Fetches ALL the data needed to load the graphs
# This is only called on the page initial load
# Populates the global 'data' variable with all the data
def getData():
  global data
  for sensor in sensors:
    data[sensor] = {}
    dfTemps,dfHumids,dfPressures,dfLux = pickData(sensor, True)
    data[sensor]["Temperature"] = dfTemps
    data[sensor]["Humidity"] = dfHumids
    data[sensor]["Pressure"] = dfPressures
    data[sensor]["Lux"] = dfLux
# getData()

#### `getNewData()` - Gets Most Recent Data

Only returns the most recent of each type of reading. Will only add to `data` if it doesn't already exist.

In [ ]:
# Runs based on the interval callback
# Gets the most recent data for each sensor
# If it's newer than what's currently in 'data', add it to 'data'
def getNewData():
  global data
  for sensor in sensors:
    dfTemps,dfHumids,dfPressures,dfLux = pickData(sensor,False)
    if dfTemps['captured_at'].iloc[0] > data[sensor]['Temperature']['captured_at'].iloc[-1]:
      data[sensor]['Temperature'] = data[sensor]['Temperature'].append(dfTemps)
      print("new Temperature")
    if dfHumids['captured_at'].iloc[0] > data[sensor]["Humidity"]["captured_at"].iloc[-1]:
      data[sensor]["Humidity"] = data[sensor]["Humidity"].append(dfHumids)
      print("new Humidity")
    if dfPressures['captured_at'].iloc[0] > data[sensor]["Pressure"]["captured_at"].iloc[-1]:
      data[sensor]["Pressure"] = data[sensor]["Pressure"].append(dfPressures)
      print("new Pressure")
    if dfLux['captured_at'].iloc[0] > data[sensor]["Lux"]["captured_at"].iloc[-1]:
      data[sensor]["Lux"] = data[sensor]["Lux"].append(dfLux)
      print("new Lux")
  return data
# getNewData()

### Plots

#### Per-Sensor Plot

This plot shows each Reading Type from one sensor and also draws the corresponding reading from Outside.

You can zoom on all plots in this demo by clicking and dragging. (not here through, this is just an image)

![](https://i.ibb.co/Fmkgdd3/Screen-Shot-2021-09-24-at-4-35-58-PM.png)

In [ ]:
# Four stacked line plots each representing a `Reading_Type` from the selected sensor
def drawPlotForSensor(sensor="Workshop"):
  print(sensor)
  fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.02)
  # Just add in all the data
  # Temperature
  fig.add_trace(go.Scatter(x=data[sensor]["Temperature"]["captured_at"], y=data[sensor]["Temperature"]["value"], mode="lines", name="Temperature"), row=1, col=1)
  fig.add_trace(go.Scatter(x=data["Outside"]["Temperature"]["captured_at"], y=data["Outside"]["Temperature"]["value"], mode="lines", name="Outside Temperature"), row=1, col=1)
  # Humidity
  fig.add_trace(go.Scatter(x=data[sensor]["Humidity"]["captured_at"], y=data[sensor]["Humidity"]["value"], mode="lines", name="Humidity"), row=2, col=1)
  fig.add_trace(go.Scatter(x=data["Outside"]["Humidity"]["captured_at"], y=data["Outside"]["Humidity"]["value"], mode="lines", name="Outside Humidity"), row=2, col=1)
  # Pressure
  fig.add_trace(go.Scatter(x=data[sensor]["Pressure"]["captured_at"], y=data[sensor]["Pressure"]["value"], mode="lines", name="Pressure"), row=3, col=1)
  fig.add_trace(go.Scatter(x=data["Outside"]["Pressure"]["captured_at"], y=data["Outside"]["Pressure"]["value"], mode="lines", name="Outside Pressure"), row=3, col=1)
  # Light
  fig.add_trace(go.Scatter(x=data[sensor]["Lux"]["captured_at"], y=data[sensor]["Lux"]["value"], mode="lines", name="Brightness"), row=4, col=1)
  
  # Add Axis Labels to charts
  fig.update_layout(
      title=sensor + " Sensor Readings",
      yaxis_title="Temperature (C)",
      yaxis2_title="Humidity (%)",
      yaxis3_title="Pressure (mbar)",
      yaxis4_title="Light Intensity (lux)",
      xaxis4_title="Datetime",
  )

  # Add Background colors to Light Graph
  fig.update_layout(
      height=800,
      shapes=[
        dict(type="rect", xref ="x domain", yref="y4", x0=0, y0=0.1, x1=1, y1=10, layer="below", fillcolor="lightgrey", line=dict(width=0), ),
        dict(type="rect", xref ="x domain", yref="y4", x0=0, y0=10, x1=1, y1=100, layer="below", fillcolor="lightyellow", line=dict(width=0)),
        dict(type="rect", xref ="x domain", yref="y4", x0=0, y0=100, x1=1, y1=1000, layer="below", fillcolor="lightblue", line=dict(width=0)),
        dict(type="rect", xref ="x domain", yref="y4", x0=0, y0=1000, x1=1, y1=100000, layer="below", fillcolor="gold", line=dict(width=0)),
        dict(type="line", xref ="x domain", yref="y1", x0=0, y0=20.55, x1=1, y1=20.55, layer="below", fillcolor="gold", line=dict(width=3, dash="dashdot", color="steelblue"), name="AC Temp"),
      ])
  # Lux is logarithmic
  fig.update_yaxes(type="log", row=4, col=1)
  fig.update_layout(uirevision='constant')
  return fig
# drawPlotForSensor("Window")


#### Sensor Comparison Plot

This plot allows us to compare a given reading type across both sensors and the outdoors. It only shows one reading type at a time, but that will be controlled by a dropdown

![](https://i.ibb.co/8Y6xfNN/Screen-Shot-2021-09-24-at-4-42-38-PM.png)

In [ ]:
# Shows the correspondance between the readings of both sensors and the outdoors
def drawOverlapPlot(measurement):
  firstOverlap = pd.Timestamp(0, unit='s')
  mostRecent = data["Window"][measurement]["captured_at"].iloc[-1]
  
  for sensor in sensors:
    firstReading = data[sensor][measurement]["captured_at"].iloc[0]
    if firstReading > firstOverlap:
      firstOverlap = firstReading

  fig = go.Figure()

  fig.add_trace(go.Scatter(x=data["Window"][measurement]["captured_at"], y=data["Window"][measurement]["value"], mode="lines", name="Window"))
  fig.add_trace(go.Scatter(x=data["Workshop"][measurement]["captured_at"], y=data["Workshop"][measurement]["value"], mode="lines", name="Workshop"))
  if measurement != "Lux":
    fig.add_trace(go.Scatter(x=data["Outside"][measurement]["captured_at"], y=data["Outside"][measurement]["value"], mode="lines", name="Outside"))

  fig.update_layout(
      height=800,
      xaxis_range=[firstOverlap, mostRecent]
  )
  fig.update_layout(uirevision='constant')

  if measurement == "Lux":
    fig.update_layout(
      shapes=[
        dict(type="rect", xref ="x domain", yref="y", x0=0, y0=0.1, x1=1, y1=10, layer="below", fillcolor="lightgrey", line=dict(width=0), ),
        dict(type="rect", xref ="x domain", yref="y", x0=0, y0=10, x1=1, y1=100, layer="below", fillcolor="lightyellow", line=dict(width=0)),
        dict(type="rect", xref ="x domain", yref="y", x0=0, y0=100, x1=1, y1=1000, layer="below", fillcolor="lightblue", line=dict(width=0)),
        dict(type="rect", xref ="x domain", yref="y", x0=0, y0=1000, x1=1, y1=100000, layer="below", fillcolor="gold", line=dict(width=0)),
      ])
    fig.update_yaxes(type="log")
  elif measurement == "Temperature":
    fig.update_layout(
    shapes=[
      dict(type="line", xref ="x domain", yref="y1", x0=0, y0=20.55, x1=1, y1=20.55, layer="below", fillcolor="gold", line=dict(width=3, dash="dashdot", color="steelblue"), name="AC Temp"),
    ])

  return fig

# drawOverlapPlot("Temperature")

#### Room View

The Room View / Sensor Overview shows the "Digital Twin" view of the room and workshop along with the most recent reading from each corresponding sensor.

The lighting within the room will change based on what is sensed and in later parts of the tutorial, indicators will show when certain conditions are detected such as the AC being on, or the windows being open.

![](https://i.ibb.co/bz94sq3/Screen-Shot-2021-09-24-at-4-45-18-PM.png)

In [ ]:
# CSS styling for the placement of the room images
# They need to stack and we'll set their opacity dynamically based on our readings below
ROOMIMAGE_STYLE = {
    "width": "100%",
    "maxWidth": "850px",
    "position": "absolute",
    "top": "0px",
    "left": "0px",
    "margin": "auto",
}

def updateRoom():
  # Get the most recent light level
  roomLightLevel = data["Window"]["Lux"]["value"].iloc[-1]
    # Set the Lights
  if roomLightLevel > 1000:
    windowOpacity = math.log(roomLightLevel,10)-2.9
    lightsOpacity = 0.0
    cloudsOpacity = 0.0
  elif roomLightLevel > 10 and roomLightLevel <= 100:
    lightsOpacity = 1.0
    windowOpacity = 0.0
    cloudsOpacity = 0.0
  elif roomLightLevel > 100 and roomLightLevel <= 1000:
    lightsOpacity = 0.0
    windowOpacity = 0.0
    cloudsOpacity = math.log(roomLightLevel,10)-1.9
  else:
    windowOpacity = 0.0
    lightsOpacity = 0.0
    cloudsOpacity = 0.0

  workshopLightLevel = data["Workshop"]["Lux"]["value"].iloc[-1]
    # Set the Lights
  if workshopLightLevel > 1000:
    workshopWindowOpacity = math.log(workshopLightLevel,10)-2.8
    workshopLightsOpacity = 0.0
    workshopCloudsOpacity = 0.0
  elif workshopLightLevel > 10 and workshopLightLevel <= 100:
    workshopLightsOpacity = 1.0
    workshopWindowOpacity = 0.0
    workshopCloudsOpacity = 0.0
  elif workshopLightLevel > 100 and workshopLightLevel <= 1000:
    workshopLightsOpacity = 0.0
    workshopWindowOpacity = 0.0
    workshopCloudsOpacity = math.log(workshopLightLevel,10)-1.8
  else:
    workshopWindowOpacity = 0.0
    workshopLightsOpacity = 0.0
    workshopCloudsOpacity = 0.0

  # Each `Img` is a "layer" in the final room view. Lighting drives the opacity of the different lighting scenarios.
  room = [
    html.Img(src="https://github.com/DanBarkus/IoT-Dashboard/blob/main/images/Both/Room_Dark.png?raw=true", id="room", className="roomImage", style=ROOMIMAGE_STYLE),
    html.Img(src="https://github.com/DanBarkus/IoT-Dashboard/blob/main/images/Both/Workshop_Dark.png?raw=true", id="workshop", className="roomImage", style=ROOMIMAGE_STYLE),
    html.Img(src="https://github.com/DanBarkus/IoT-Dashboard/blob/main/images/Both/Room_Lights.png?raw=true", id="lights", className="roomImage", style={**ROOMIMAGE_STYLE, **{"opacity":lightsOpacity}}),
    html.Img(src="https://github.com/DanBarkus/IoT-Dashboard/blob/main/images/Both/Workshop_Lights.png?raw=true", id="workshop-lights", className="roomImage", style={**ROOMIMAGE_STYLE, **{"opacity":workshopLightsOpacity}}),
    html.Img(src="https://github.com/DanBarkus/IoT-Dashboard/blob/main/images/Both/Room_Cloudy.png?raw=true", id="cloudy", style={**ROOMIMAGE_STYLE, **{"opacity":cloudsOpacity}}),
    html.Img(src="https://github.com/DanBarkus/IoT-Dashboard/blob/main/images/Both/Workshop_Cloudy.png?raw=true", id="workshop-cloudy", style={**ROOMIMAGE_STYLE, **{"opacity":workshopCloudsOpacity}}),
    html.Img(src="https://github.com/DanBarkus/IoT-Dashboard/blob/main/images/Both/Room_Sun.png?raw=true", id="sun", style={**ROOMIMAGE_STYLE, **{"opacity":windowOpacity}}),
    html.Img(src="https://github.com/DanBarkus/IoT-Dashboard/blob/main/images/Both/Workshop_Sun.png?raw=true", id="workshop-sun", style={**ROOMIMAGE_STYLE, **{"opacity":workshopWindowOpacity}})
  ]

  # These are just the tables that hold the most recent readings for each sensor
  windowStats = dbc.Table([
    html.Thead([html.Tr([html.Th("Reading Type"), html.Th("Reading")])]),
    html.Tr([html.Td("Temperature"), html.Td(str(data["Window"]["Temperature"]["value"].iloc[-1]) + "℃")]),
    html.Tr([html.Td("Humidity"), html.Td(str(data["Window"]["Humidity"]["value"].iloc[-1]) + "%")]),
    html.Tr([html.Td("Pressure"), html.Td(str(round(data["Window"]["Pressure"]["value"].iloc[-1],2)) + " mbar")]),
    html.Tr([html.Td("Light Level"), html.Td(str(data["Window"]["Lux"]["value"].iloc[-1]) + " lux")]),
    html.Tr([html.Td("Most Recent"), html.Td(str(data["Window"]["Lux"]["captured_at"].iloc[-1]))]),
  ],
  bordered=True,
  size="sm")

  workshopStats = dbc.Table([
    html.Thead([html.Tr([html.Th("Reading Type"), html.Th("Reading")])]),
    html.Tr([html.Td("Temperature"), html.Td(str(data["Workshop"]["Temperature"]["value"].iloc[-1]) + "℃")]),
    html.Tr([html.Td("Humidity"), html.Td(str(data["Workshop"]["Humidity"]["value"].iloc[-1]) + "%")]),
    html.Tr([html.Td("Pressure"), html.Td(str(round(data["Workshop"]["Pressure"]["value"].iloc[-1],2)) + " mbar")]),
    html.Tr([html.Td("Light Level"), html.Td(str(data["Workshop"]["Lux"]["value"].iloc[-1]) + " lux")]),
    html.Tr([html.Td("Most Recent"), html.Td(str(data["Workshop"]["Lux"]["captured_at"].iloc[-1]))]),
  ],
  bordered=True,
  size="sm")

  return room,windowStats,workshopStats

### The Rest of the Page

Now that the charts are taken care of, it's time to build the actual page(s) and thier components.

#### Sidebar

The sidebar will serve as the navigation between our pages and just hold some additional information about the Site as well.

In [ ]:
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "backgroundColor": "#424242"
}
TG_LOGO = "https://res.cloudinary.com/crunchbase-production/image/upload/c_lpad,h_170,w_170,f_auto,b_white,q_auto:eco,dpr_1/vootpjtmcyx0gt79zl6r"

sidebar = html.Div(
    [
        # A brief description 

        html.Center(html.H1(
            "IoT Digital Twin for interior Micro-Climates", className="lead", style={'color':"#FF6D01"}
        )),
        html.Br(), 
     
        # The navbar itself
        dbc.Nav(
            [
                dbc.NavLink("Per-sensor Readings", href="/", active="exact", style={'color':'white'}),
                dbc.NavLink("Sensor Comparisons", href="/comparison", active="exact", style={'color':'white'}),
            ],
            vertical=True,
            pills=True,
        ),
     
        html.Br(), 
        html.Br(), 
     
        # The TigerGraph logo as well as a link to TG Cloud
     
        html.Center(dbc.Row(dbc.Col(html.Img(src=TG_LOGO, height="150px", style={"marginBottom": "15px"})))),
        html.Center(html.B(html.A("TigerGraph Cloud", href="https://www.tigergraph.com/cloud/", target="_blank", style={'color':'white'}))),

    ],
    style=SIDEBAR_STYLE,
)

#### Chart Holders and Dropdowns

General structure for the room view. 

Stats - Room - Stats

In [ ]:
roomView = dbc.Row([
  dbc.Col([
    html.H3(["Workshop"]),
    html.Div(
        id="workshop-stats",
        style={"display":"flex", "flexDirection":"column"})
  ], width=3),
  # Holds the room image
  dbc.Col(
      id='room-container',
  style={"position": "relative", "width": "800px", "height": "450px", "flex": "none"},
  width=6),
  # Holds the current Stats
  dbc.Col([
    html.H3(["Window"]),
    html.Div(
        id="window-stats",
        style={"display":"flex", "flexDirection":"column"})
  ], width=3)
  ],style={"display":"flex", "alignItems": "center", "justifyContent":"space-around"})

In [ ]:
# Holds the per sensor graph
sensorsView = dbc.Row([
    dbc.Col([
      html.H3("Window", id="sensor-display"),
      dbc.DropdownMenu(
          [dbc.DropdownMenuItem(x, id=x) for x in sensors if x != "Outside"],
          id='dropdown-location',
          label="Select Sensor",
          bs_size='lg'
      ),
      dcc.Loading(children=[
          dcc.Graph(id="graph"),
      ],
      type="graph",
      )
    ],
    width=12
    ),
  ])

In [ ]:
# Holds the comparison Graph
comparisonView = html.Div([
    html.H3("Temperature", id="type-display"),
    dbc.DropdownMenu(
        [dbc.DropdownMenuItem(x, id=x) for x in readingTypes],
        id='dropdown-type', 
        label='Select Reading Type'
    ),
    dcc.Loading(children=[
        dcc.Graph(id="comparison-graph"),
    ],
    type="graph",
    )
  ],
  style={"alignSelf":"flex-end", "width":"100%"}
  )

### The Dash App Itself

This is where everything comes together. All of our components are in their place and the only thing left to do is declare the Dash app itself and its callbacks.

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY], suppress_callback_exceptions=True)

getData()

app.layout = html.Div([
    # Data store
    dcc.Store(id='graph-data'),
    dcc.Interval(
        id='interval-component',
        interval=120*1000, # in milliseconds
        n_intervals=0
    ),
    dcc.Location(id="url"),
    # Main wrapper div
    dbc.Container([
        sidebar,
        roomView,
        html.Div([],id="content-holder"),
    ],
    fluid=True,
    style={"paddingLeft":"17rem"}),
])   
app.scripts.config.serve_locally = False

# -----------------------------
# ----- Here be Callbacks -----
# -----------------------------

# Navigation callback
@app.callback(Output("content-holder", "children"), 
              [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
      return sensorsView 
    elif pathname == "/comparison":
      return comparisonView
    elif pathname == "/page-2":
      return personContent 
    # If the user tries to reach a different page, return an error!
    return dbc.Jumbotron([
      html.H1("404: Not found", className="text-danger"),
      html.Hr(),
      html.P("Uh oh! Unfortunately, the pathname {pathname} was unable to be recognised..."),
    ])

# Sensor select callback (also triggers on data update)
@app.callback(
    [Output("graph", "figure"),
     Output("sensor-display", "children")],
    [Input("Window", "n_clicks"),
     Input("Workshop", "n_clicks"),
     Input('graph-data', 'data')])
def update_graph_sensor(n1,n2,d):
    global currentSensor
    ctx = dash.callback_context
    if not ctx.triggered:
      btn_id = "Window"
    else:
      btn_id = ctx.triggered[0]["prop_id"].split(".")[0]
      if btn_id == "graph-data":
        btn_id = currentSensor
    currentSensor = btn_id
    fig = drawPlotForSensor(btn_id)
    return fig, btn_id

# Reading select callback (also triggers on data update)
@app.callback(
    [Output("comparison-graph", "figure"),
     Output("type-display", "children")],
    [Input("Temperature", "n_clicks"),
     Input("Humidity", "n_clicks"),
     Input("Pressure", "n_clicks"),
     Input("Lux", "n_clicks"),
     Input('graph-data', 'data')
     ])
def update_graph_comparison(n1,n2,n3,n4,d):
    global currentType
    ctx = dash.callback_context
    if not ctx.triggered:
      btn_id = "Temperature"
    else:
      btn_id = ctx.triggered[0]["prop_id"].split(".")[0]
      if btn_id == "graph-data":
        btn_id = currentType
    currentType = btn_id
    fig = drawOverlapPlot(btn_id)
    return fig, btn_id

# Room update on data update
@app.callback(
    [Output('room-container', 'children'),
     Output('window-stats', 'children'),
     Output('workshop-stats', 'children')],
     Input('graph-data','data'))
def update_room(d):
    room,windowStats,workshopStats = updateRoom()
    return room,windowStats,workshopStats

# Interval callback to check for new data every minute
@app.callback(
    Output('graph-data', 'data'), 
    Input('interval-component', 'n_intervals'),
    State('graph-data', 'data'), prevent_initial_call=True)
def update_data(n_intervals, dat):
    getNewData()
    return {}


app.run_server(mode="external", port=5051, debug=True, threaded=True)

In [ ]:
conn.gsql('''
CREATE SCHEMA_CHANGE JOB add_next_reading FOR GRAPH IoTDashboard { 
  ADD DIRECTED EDGE next_reading(FROM Reading, TO Reading, seconds_between FLOAT, delta_value FLOAT) WITH REVERSE_EDGE="reverse_next_reading";
}
''')

In [ ]:
conn.gsql('''
RUN SCHEMA_CHANGE JOB add_next_reading
DROP JOB add_next_reading
''')

In [ ]:
for sensor in sensors:
  for typ in readingTypes:
    if sensor == 'Outside' and typ == 'LUX':
      typ = "CLOUD"
    conn.runInstalledQuery('addNextEdge', {'inDevice': sensor, 'inReading':typ.upper()})

In [ ]:
for sensor in sensors:
  res = conn.runInstalledQuery("exportData", params={"inDevice": sensor})
  df = conn.vertexSetToDataFrame(res[0]["deviceReadings"])
  df = df.sort_values("captured_at")
  df = df.drop(columns=['v_id', 'id'])
  df = df.reindex(columns=['@device', '@rdgType', 'value', 'type', 'captured_at'])
  df.to_csv(sensor + '.csv')

Generate Edges between readings

In [ ]:
conn.gsql(
    '''
    USE GRAPH IoTDashboard
    CREATE QUERY addNextEdge(VERTEX<Device> inDevice, VERTEX<Reading_Type> inReading) FOR GRAPH IoTDashboard SYNTAX v2{ 
      TYPEDEF TUPLE<DATETIME date, VERTEX<Reading> reading> readingOrder;
      VERTEX<Reading> prevReading;
      VERTEX<Reading> currentReading;
      FLOAT timeDelta;
      FLOAT valueDelta;
      SetAccum<VERTEX<Reading>> @@readings;
      HeapAccum<readingOrder>(10, date ASC) @@allReadings;
      device = {inDevice};
      rdgType = {inReading};
      readings = {Reading.*};
      
      @@allReadings.resize(readings.size());

      readings = SELECT r FROM device - () - Reading:r - () - rdgType
        WHERE
          r.outdegree("next_reading") == 0
        ACCUM
          @@readings += r
        POST-ACCUM
          @@allReadings += readingOrder(r.captured_at, r);
      
      @@allReadings.resize(readings.size());
      
      PRINT @@allReadings;
      
      FOREACH reading in @@readings DO
        IF @@allReadings.size() > 1 THEN
          prevReading = @@allReadings.pop().reading;
          currentReading = @@allReadings.top().reading;
          timeDelta = datetime_to_epoch(currentReading.captured_at) - datetime_to_epoch(prevReading.captured_at);
          valueDelta = currentReading.value - prevReading.value;
          INSERT INTO next_reading (FROM, TO, seconds_between, delta_value)VALUES (prevReading.id, currentReading.id, timeDelta, valueDelta);
        ELSE
          BREAK;
        END;
      END;
    }
    '''
)